In [7]:
pip install mne h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 15.6 MB/s  0:00:00 eta 0:00:01

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import scipy.io as sio
import numpy as np

# Load MATLAB NIRS file
mat_data = sio.loadmat('S133_run03.nirs')

# Extract key data
data = mat_data['d']  # Time-series data (16875 x 48 channels)
time = mat_data['t'].flatten()  # Time vector
ml = mat_data['ml']  # Measurement list
sd = mat_data['SD'][0, 0]  # Source-detector configuration
info = mat_data['systemInfo'][0, 0]  # System info

print("NIRS Data loaded successfully!")
print(f"Data shape: {data.shape}")
print(f"Time points: {len(time)}")
print(f"Measurement list shape: {ml.shape}")
print(f"Duration: {time[-1] - time[0]:.2f} seconds")
print(f"Sampling rate: {1 / np.mean(np.diff(time)):.1f} Hz")

NIRS Data loaded successfully!
Data shape: (16875, 48)
Time points: 16875
Measurement list shape: (48, 4)
Duration: 674.96 seconds
Sampling rate: 25.0 Hz


In [7]:
import scipy.io as sio

# Load the MATLAB file
data = sio.loadmat('S133_run03.nirs')
print("Keys in MATLAB file:")
for key in data.keys():
    if not key.startswith('__'):
        print(f"  - {key}: {type(data[key]).__name__}")
        if hasattr(data[key], 'shape'):
            print(f"    Shape: {data[key].shape}")

Keys in MATLAB file:
  - SD: ndarray
    Shape: (1, 1)
  - ml: ndarray
    Shape: (48, 4)
  - d: ndarray
    Shape: (16875, 48)
  - t: ndarray
    Shape: (16875, 1)
  - dStd: ndarray
    Shape: (0, 0)
  - aux: ndarray
    Shape: (16875, 8)
  - s: ndarray
    Shape: (16875, 1)
  - tdml: ndarray
    Shape: (48, 16875)
  - systemInfo: ndarray
    Shape: (1, 1)


In [10]:
ml = mat_data['ml'] 
print("Measurement list (first 10 rows):")
print(ml[:10])

Measurement list (first 10 rows):
[[2 1 1 1]
 [2 2 1 1]
 [2 3 1 1]
 [2 4 1 1]
 [1 1 1 1]
 [1 2 1 1]
 [3 5 1 1]
 [3 7 1 1]
 [3 8 1 1]
 [4 5 1 1]]


In [12]:
## [source index, detector index, wavelength, and Data type]

In [13]:
sd = mat_data['SD'][0,0] 
print("Number of sources:", sd['nSrcs'][0][0]) 
print("Number of detectors:", sd['nDets'][0][0]) 
print("Source positions:\n", sd['SrcPos']) 
print("Detector positions:\n", sd['DetPos'])

Number of sources: 8
Number of detectors: 16
Source positions:
 [[100 120   0]
 [ 70  90   0]
 [ 70  40   0]
 [ 40  10   0]
 [120 120   0]
 [150  90   0]
 [150  40   0]
 [180  10   0]]
Detector positions:
 [[ 70 120   0]
 [100  90   0]
 [ 40  90   0]
 [ 70  60   0]
 [ 40  40   0]
 [ 10  10   0]
 [ 70  10   0]
 [100  40   0]
 [150 120   0]
 [120  90   0]
 [180  90   0]
 [150  60   0]
 [180  40   0]
 [210  10   0]
 [150  10   0]
 [120  40   0]]


In [14]:
for key in mat_data.keys(): 
    if "aux" in key.lower(): 
        print("Found AUX key:", key, "shape:", mat_data[key].shape)

Found AUX key: aux shape: (16875, 8)


In [15]:
aux = mat_data['aux']

In [16]:
aux

array([[0.04714678, 0.01953006, 0.04958804, ..., 0.04577357, 0.04577357,
        0.04577357],
       [0.04638389, 0.01876717, 0.04882514, ..., 0.04562099, 0.04592615,
        0.04592615],
       [0.04394263, 0.01602075, 0.04623131, ..., 0.04562099, 0.0448581 ,
        0.04546842],
       ...,
       [0.04409521, 0.01663106, 0.04638389, ..., 0.04577357, 0.04562099,
        0.04592615],
       [0.04440037, 0.01693622, 0.04668904, ..., 0.04623131, 0.04546842,
        0.04531584],
       [0.04729936, 0.01953006, 0.04958804, ..., 0.04546842, 0.04592615,
        0.04577357]], shape=(16875, 8))

In [17]:
# Threshold to detect triggers 
trigger = aux[:, 0] > np.mean(aux[:, 0]) + 2*np.std(aux[:, 0]) 
# Convert boolean → int 
S = trigger.astype(int).reshape(-1, 1) 
print("S matrix shape:", S.shape)

S matrix shape: (16875, 1)


In [18]:
import numpy as np 
aux = mat_data["aux"] 
# shape: (timepoints, aux_channels) 
# # Create an empty stimulus matrix with one column per AUX channel 
S = np.zeros_like(aux) 
# Convert each AUX channel into a binary stimulus vector 
for i in range(aux.shape[1]): 
    channel = aux[:, i] 
    threshold = np.mean(channel) + 2 * np.std(channel) 
    S[:, i] = (channel > threshold).astype(int) 
print("Stimulus matrix S shape:", S.shape)

Stimulus matrix S shape: (16875, 8)


In [22]:
import mne 
import numpy as np 
# Build MNE channel names in the expected format 
ch_names = [] 
ch_types = [] 
for src, det, wl, dtype in ml: 
    ch_names.append(f"S{int(src)}_D{int(det)} {int(wl)}") 
    ch_types.append("fnirs_cw_amplitude") # REQUIRED for optical_densit

In [23]:
sfreq = 1 / np.mean(np.diff(time)) 
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

/tmp/ipykernel_7001/1711137860.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'S2_D3 1', 'S6_D12 1', 'S8_D14 1', 'S4_D7 1', 'S6_D10 1', 'S5_D10 1', 'S7_D13 1', 'S1_D2 1', 'S4_D5 1', 'S2_D2 1', 'S6_D11 1', 'S2_D4 1', 'S2_D1 1', 'S3_D8 1', 'S7_D16 1', 'S6_D9 1', 'S8_D15 1', 'S4_D6 1', 'S5_D9 1', 'S3_D5 1', 'S8_D13 1', 'S7_D15 1', 'S1_D1 1', 'S3_D7 1'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)


In [20]:
import mne
import numpy as np
from mne.preprocessing.nirs import optical_density, beer_lambert_law

# use mat_data if available else fallback to data
mat = mat_data if 'mat_data' in globals() else data

# prepare raw data: channels x times
raw_data = mat['d'].T.astype(float)

# sampling frequency from time vector
time_vec = mat['t'].flatten()
sfreq = 1.0 / np.mean(np.diff(time_vec))

# create channel info (fnirs CW amplitude)
ch_names = [f'Ch{i+1}' for i in range(raw_data.shape[0])]
ch_types = ['fnirs_cw_amplitude'] * raw_data.shape[0]
info = mne.create_info(ch_names, sfreq, ch_types=ch_types)

raw = mne.io.RawArray(raw_data, info)

# convert to optical density and then to haemoglobin concentration
raw_od = optical_density(raw)
raw_hb = beer_lambert_law(raw_od)

Creating RawArray with float64 data, n_channels=48, n_times=16875
    Range : 0 ... 16874 =      0.000 ...   674.960 secs
Ready.


AttributeError: 'NoneType' object has no attribute 'groups'